In [1]:
# Instead of doing loss.bckward we try to implement the backpropogation manually

In [34]:
# Referring https://www.youtube.com/watch?v=q8SA3rM6ckI&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=8 

In [36]:
# to get the derivatives use https://www.wolframalpha.com/

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read all the words 
words = open("names.txt", 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [4]:
# build the vocabulary of words
chars  = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [5]:
# build data set
block_size = 3 # number of inputs passed to predict the next character

def build_dataset(words):
    X, Y = [],  []
    for w in words:
        context = [0]*block_size
        for ch in w+'.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

# sample dataset into 80% train, 10% validation and 10% test

import random
random.seed(42)
random.shuffle(words)
n1 = (int)(0.8 * len(words))
n2 = (int)(0.9 * len(words))

Xtr, Ytr   = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xte , Yte  = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [20]:
# utility function that we use to compare the gradients we calculated manualy and the gradients generated by pytorch
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact : {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [21]:
# neural Network initializations
n_embd = 10 # input characters will be converted to 10D embedded vectors
n_hidden = 64 # the number of neurins in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducability
C = torch.randn((vocab_size, n_embd),           generator = g)

# layer 1
W1 = torch.randn((n_embd*block_size, n_hidden), generator = g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                      generator = g) * 0.1 # using b1 just to see how backpropogations do onthis. We dont have to use htis if we use batch norm

# layer 2
W2 = torch.randn((n_hidden, vocab_size),        generator = g) * 0.1
b2 = torch.randn((vocab_size),                  generator = g) * 0.1

# batch norm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: Initializing parameters in nonstandard ways because sometimes intiilizing with Zeros could mask an incorrect implementation of backward pass

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [22]:
batch_size = 32
n = batch_size # a shortter variable for convience

# construct mini batch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator = g)
Xb, Yb = Xtr[ix], Ytr[ix] #batch X, y

In [63]:
# forward pass , chunckated into smaller steps that are possible to backward pass one at a time

emb = C[Xb]   # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatinate hte vectors

# linear layer
hprebn = embcat @ W1 + b1 # hidden layer pre activation

# Batch normalization :    hpreact = bngain * (hpreact - hpreact.mean(0, keepdims = True))/hpreact.std(0,keepdim = True) + bnbias
# we split this into chunks so the backpropagation is easy

bnmeani = 1/n * hprebn.sum(0, keepdim = True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim = True) # note : Bessels correction . Dividing by n-1 not n
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# non linearity
h = torch.tanh(hpreact) # hidden layer

# linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy
logit_maxes = logits.max(1, keepdim = True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim = True)
counts_sum_inv = counts_sum**-1 # using this instead of 1/counts_sum so we can backpropagate easily
probs = counts* counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, 
          logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff, bndiff2, hprebn, bnmeani, embcat, emb]:
    t.retain_grad()

loss.backward()
loss

tensor(3.3534, grad_fn=<NegBackward0>)

In [64]:
# backpropogation manually

In [65]:
# derivative of logprobs. log probs is impacted by loss 
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n

# derivative of probs
dprobs = (1.0 / probs) * dlogprobs

# derivative of counts_sum_inv
# C = a * b, but with tensors:
# a[3,3] * b[3, 1] ----->
# a11*b1 a12*b1 a13*b1
# a21*b2 a22*b2 a23*b2
# a31*b3 a32*b3 a33*b3
# c[3, 3]

# size of the is [32,1] and size of counts is [32, 27] => we replicate the counts_sum_inv 
# dc/da = b & dc/db = a
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim = True) # this is derivative of replicated dcounts_sum_inv. now we want derivative for the non replicated one. this is nothing but the sum of the all the derivatives

# derivative of counts
dcounts = counts_sum_inv * dprobs # counts+sum_iv is 32,1 and dprobs i s[32,27], replication happens and will get correct result.
# counts is used in 2 operations once for probs calculation and once for counts_sum_inv calculation

#derivative of counts_sum
dcounts_sum = (-counts_sum ** -2) * dcounts_sum_inv

# derivative of counts
# the internal operation here is 
# a11+a12+a13 = b11 => we give the same gradient to all teh values a11 , a12, a13 
# now because counts is contributing in 2 operations once in probs calculation and once in counts calculation. We add up the gradient that we calucluted above
dcounts += torch.ones_like(counts) * dcounts_sum

dnorm_logits = counts * dcounts

# calculate the derivative of logits. Remember it is used in 2 operations. so we accumulate the derivative.
#logits.shape is [32,27], logits_maxes.shape is [32,1]
dlogits = dnorm_logits.clone()

dlogit_maxes = (-dnorm_logits).sum(1, keepdim = True)

# calculating the derivative of logits. i.e picking the max value
# pick the indice that has maximum value and then multiply the dlogits_maxes. For thi suse one_hot enoding.
dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1]) * dlogit_maxes

dh = dlogits @ W2.T

dW2 = h.T @ dlogits

db2 = dlogits.sum(0)

dhpreact = (1.0 - h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim = True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim = True)
dbndiff = dbnraw * bnvar_inv
dbnvar_inv = (dbnraw * bndiff).sum(0, keepdim = True)
dbnvar = ( -0.5 * (bnvar + 1e-5) ** -1.5) * dbnvar_inv
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
dbndiff += (2*bndiff) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]
    
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact : True  | approximate: True  | maxdiff: 0.0
probs           | exact : True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact : True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact : True  | approximate: True  | maxdiff: 0.0
counts          | exact : True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact : True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact : True  | approximate: True  | maxdiff: 0.0
logits          | exact : True  | approximate: True  | maxdiff: 0.0
h               | exact : True  | approximate: True  | maxdiff: 0.0
W2              | exact : True  | approximate: True  | maxdiff: 0.0
b2              | exact : True  | approximate: True  | maxdiff: 0.0
hpreact         | exact : True  | approximate: True  | maxdiff: 0.0
bngain          | exact : True  | approximate: True  | maxdiff: 0.0
bnbias          | exact : True  | approximate: True  | maxdiff: 0.0
bnraw           | exact : True  | approximate: T

In [67]:
# Now we optimize the backpropogation steps. We can do this by minimizing the calculations of loss and batchnormalization

# backpropogation
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

# because of the floating point variations we might not get exact result but we will get approximate result
cmp('logits', dlogits, logits)

#batch normalization
dhprebn = bngain * bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw *(dhpreact*bnraw).sum(0))
cmp('hprebn', dhprebn , hprebn)

logits          | exact : False | approximate: True  | maxdiff: 6.05359673500061e-09
hprebn          | exact : False | approximate: True  | maxdiff: 9.313225746154785e-10


In [72]:
# put it all together

#init
n_embd = 10 # the dimensionality of the character embedding vector
n_hidden = 20 # the number of neuroins in the hidden layer

g = torch.Generator().manual_seed(2147483647)
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 100000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())


1497
      0/ 100000: 3.3000
  10000/ 100000: 2.7611
  20000/ 100000: 2.1759
  30000/ 100000: 2.3806
  40000/ 100000: 2.2181
  50000/ 100000: 2.1695
  60000/ 100000: 1.9883
  70000/ 100000: 2.5655
  80000/ 100000: 1.9129
  90000/ 100000: 1.9810


In [73]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [74]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xval, Yval),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.246192216873169
val 2.2526233196258545


In [75]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carlah.
amilllei.
kimlix.
taty.
halans.
ejrahnen.
den.
ric.
kaeli.
nellaiah.
maiiv.
kaleig.
halmondin.
quinn.
sulin.
alian.
qui.
jero.
dearyxi.
fraen.
